In [20]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, RandomTranslation, RandomZoom, RandomRotation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPool2D, Dense, Dropout
from timeit import default_timer as timer


In [21]:
train_full = pd.read_csv('data/sign_mnist_train.csv')
test = pd.read_csv('data/sign_mnist_test.csv')

In [22]:
label_map = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [23]:
X_train_full = train_full.values[:,1:].astype(np.float32())
y_train_full = train_full.values[:,0]
X_test = test.values[:,1:].astype(np.float32())
y_test = test.values[:,0]

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=2500)
for train_index, test_index in split.split(X_train_full, y_train_full):
    X_train, X_val = X_train_full[train_index], X_train_full[test_index]
    y_train, y_val = y_train_full[train_index], y_train_full[test_index]

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [7]:
X_train = X_train.reshape((X_train.shape[0],28,28,1))
X_val = X_val.reshape((X_val.shape[0],28,28,1))
X_test = X_test.reshape((X_test.shape[0],28,28,1))
image_shape = X_train[0].shape

In [8]:
model = tf.keras.Sequential([
    Input(shape=image_shape),
    RandomTranslation(0.15, 0.15),
    RandomZoom(0.15),
    RandomRotation(0.1),
    Conv2D(8, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    Conv2D(8, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=2, strides=2, padding="valid"),
    Conv2D(16, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    Conv2D(16, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=2, strides=2, padding="valid"),
    Conv2D(32, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    Conv2D(32, kernel_size=(3,3), strides=1, padding="same"),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=2, strides=2, padding="valid"),
    Flatten(),
    Dense(100),
    Dropout(0.50),
    ReLU(),
    Dense(26, activation='softmax')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_translation (RandomT  (None, 28, 28, 1)        0         
 ranslation)                                                     
                                                                 
 random_zoom (RandomZoom)    (None, 28, 28, 1)         0         
                                                                 
 random_rotation (RandomRota  (None, 28, 28, 1)        0         
 tion)                                                           
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 8)         80        
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 8)        32        
 ormalization)                                                   
                                                        

In [9]:
# tf.keras.utils.plot_model(model, show_shapes=True)

In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0005), metrics=["accuracy"])

In [15]:
batch_size = 32
start_train = timer()
history = model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_data=(X_val, y_val))
end_train = timer()

Epoch 1/30
 30/780 [>.............................] - ETA: 59s - loss: 0.1261 - accuracy: 0.9615

KeyboardInterrupt: 

In [16]:
# CNN evaluation
# start_pred = timer()
# train_acc = model.evaluate(X_train, y_train)[1]
# end_pred = timer()
# test_acc = model.evaluate(X_test, y_test)[1]





In [17]:
# import pickle
# pickle.dump(model,open('model.pkl','wb')
model.save('model.h5')

In [18]:
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score

# print("Precision: ", precision_score(y_test, test_pred, average="micro").round(2))
# print("Recall: ", recall_score(y_test, test_pred, average="micro").round(2))
# print("F1 Score: ", f1_score(y_test, test_pred, average="micro").round(2))